In [1]:
import time
import sys
sys.path.append('../contract')
from typing import List
import base64
from base64 import b64encode as en64
from base64 import b64decode as de64
import pandas as pd
from algosdk import account, mnemonic, logic
from algosdk.future import transaction
from algosdk.v2client import algod, indexer
from pyteal import compileTeal, Mode, Approve

ALGOD_TOKEN = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
TESTNET_ALGOD_RPC = "https://testnet-api.algonode.network"
TESTNET_INDEXER_RPC = "https://testnet-idx.algonode.network"

algod_client = algod.AlgodClient(ALGOD_TOKEN, TESTNET_ALGOD_RPC)
indexer_client = indexer.IndexerClient(ALGOD_TOKEN, TESTNET_INDEXER_RPC)
sp_func = algod_client.suggested_params
on_complete_param = transaction.OnComplete.NoOpOC
    
mnemonic_1 = open("../wallet_1").read().replace(',', ' ')
mnemonic_2 = open("../wallet_2").read().replace(',', ' ')
mnemonic_3 = open("../wallet_3").read().replace(',', ' ')

alice_private_key = mnemonic.to_private_key(mnemonic_1)
alice_address = account.address_from_private_key(alice_private_key)
bob_private_key = mnemonic.to_private_key(mnemonic_2)
bob_address = account.address_from_private_key(bob_private_key)
carol_private_key = mnemonic.to_private_key(mnemonic_3)
carol_address = account.address_from_private_key(carol_private_key)

print("Alice {} Balance: {}".format(alice_address, 
                algod_client.account_info(alice_address).get('amount') / 1e6))
print("Bob   {} Balance: {}".format(bob_address, 
                algod_client.account_info(bob_address).get('amount') / 1e6))
print("Carol {} Balance: {}".format(carol_address, 
                algod_client.account_info(carol_address).get('amount') / 1e6))


Alice GZ4IJXXNRFT23E6SLUOSSUWN2LUDFQTX4F6SXF5EP27LFWTOWHPFANLYIQ Balance: 57.831993
Bob   CIK3P7U4PZBJESWQ3XFDCZDXEZ3JZLWY6XZHY4A4KEJCA5SYYIPBT6W7Y4 Balance: 15.302008
Carol K7ZJP3J7SYYNB42DPZMRY56X32HVTYOOZRA2ASE2IND4LSQKCL3CE2Z2YU Balance: 8.977


In [2]:
def submit_transaction(private_key: str, unsigned_txn: transaction.Transaction):
    signed_txn = unsigned_txn.sign(private_key)
    txid = algod_client.send_transaction(signed_txn)
    print("Signed transaction with txID: {}".format(txid))
    confirmed_txn = transaction.wait_for_confirmation(algod_client, txid, 3)
    print("Confirmed on round {}!".format(confirmed_txn['confirmed-round']))
    transaction_response = algod_client.pending_transaction_info(txid)
    return transaction_response
    
def submit_transaction_group(private_key: str, unsigned_txns: List[transaction.Transaction]):
    gid = transaction.calculate_group_id(unsigned_txns)
    signed_txns = []
    for unsigned in unsigned_txns:
        unsigned.group = gid
        signed = unsigned.sign(private_key)
        signed_txns.append(signed)
    gtxid = algod_client.send_transactions(signed_txns)
    print("Signed transaction group with gtxID: {}".format(gtxid))
    confirmed_txn = transaction.wait_for_confirmation(algod_client, gtxid, 3)
    print("Confirmed on round {}!".format(confirmed_txn['confirmed-round']))
    transaction_response = algod_client.pending_transaction_info(gtxid)
    return transaction_response

def compile_program(client, source_code):
    compile_response = client.compile(source_code)
    return base64.b64decode(compile_response['result'])

blank_program = compile_program(algod_client, compileTeal(
    Approve(), Mode.Application, version=8
))


## 1. Asset Creation and OptIn

In [2]:
# Creation
create_asset_tx = submit_transaction(alice_private_key, transaction.AssetCreateTxn(
    alice_address, sp_func(), int(1e18), 6, False, unit_name="mUSDC", asset_name="Meson Test Circle USD"
))
usdc_index = create_asset_tx['asset-index']
print("Asset [Meson Test Circle USD] created success! Asset id: %s\n" % usdc_index)

create_asset_tx = submit_transaction(alice_private_key, transaction.AssetCreateTxn(
    alice_address, sp_func(), int(1e18), 6, False, unit_name="mUSDT", asset_name="Meson Test Tether"
))
usdt_index = create_asset_tx['asset-index']
print("Asset [Meson Test Tether] created success! Asset id: %s\n" % usdt_index)

Signed transaction with txID: RXOM4DVKIEBUYX7FSPOHD2SQLPX5B6TCPHSJKS5L76XEK73UIZRA
Confirmed on round 27892638!
Asset [Meson Test Circle USD] created success! Asset id: 160363393

Signed transaction with txID: VQCAAMUL7KW5RTMRDCM6G5VRUVG4O2Q6B6KNXKB3VHNWIGODJPMQ
Confirmed on round 27892640!
Asset [Meson Test Tether] created success! Asset id: 160363405



In [3]:
# Optin
submit_transaction(bob_private_key, transaction.AssetOptInTxn(
    bob_address, sp_func(), usdc_index
)); print("Bob opt in mUSDC asset success!\n")

submit_transaction(carol_private_key, transaction.AssetOptInTxn(
    carol_address, sp_func(), usdc_index
)); print("Carol opt in mUSDC asset success!\n")

submit_transaction(bob_private_key, transaction.AssetOptInTxn(
    bob_address, sp_func(), usdt_index
)); print("Bob opt in mUSDT asset success!\n")

submit_transaction(carol_private_key, transaction.AssetOptInTxn(
    carol_address, sp_func(), usdt_index
)); print("Carol opt in mUSDT asset success!\n")

Signed transaction with txID: EWW3B5FIRFDM6WKADPHUSMM3OKCIQ5UCDKDROSAQPZJOHNANBKJA
Confirmed on round 27892658!
Bob opt in mUSDC asset success!

Signed transaction with txID: HEUXQVTMRGLTDS2ONSBRXPPQFVGFGU6FVKPQTPENV2MNNE4SLI7A
Confirmed on round 27892660!
Carol opt in mUSDC asset success!

Signed transaction with txID: O6CU4TVXM7UI5OZTCQLLY4XWNUQMXQ22RO3AWQE6OG4L7YA57GTQ
Confirmed on round 27892662!
Bob opt in mUSDT asset success!

Signed transaction with txID: U3ERKP5AH36ZZWJ7GKQCAUV6HPXAKOUUEPPCH2B6BWM45P4SRMGQ
Confirmed on round 27892664!
Carol opt in mUSDT asset success!



In [4]:
# Transfer
asset_amount = 2e12

submit_transaction(alice_private_key, transaction.AssetTransferTxn(
    alice_address, sp_func(), bob_address, int(asset_amount), usdc_index
)); print("Bob received %d mUSDC!" % (asset_amount / 1e6))

submit_transaction(alice_private_key, transaction.AssetTransferTxn(
    alice_address, sp_func(), carol_address, int(asset_amount), usdc_index
)); print("Carol received %d mUSDC!" % (asset_amount / 1e6))

submit_transaction(alice_private_key, transaction.AssetTransferTxn(
    alice_address, sp_func(), bob_address, int(asset_amount), usdt_index
)); print("Bob received %d mUSDT!" % (asset_amount / 1e6))

submit_transaction(alice_private_key, transaction.AssetTransferTxn(
    alice_address, sp_func(), carol_address, int(asset_amount), usdt_index
)); print("Carol received %d mUSDT!" % (asset_amount / 1e6))

Signed transaction with txID: LMB4X5LQBT4WFRQ7WJPN5MCGP5LUM37LTRDMKUV2FJCRJAQOWT3Q
Confirmed on round 27892666!
Bob received 2000000 mUSDC!
Signed transaction with txID: IIYMZA2RMIIG277HYAAZCZ6DGLMFYGDPJ3HXMKBYX2JGDK4ZH3HA
Confirmed on round 27892668!
Carol received 2000000 mUSDC!
Signed transaction with txID: UAY5MIOXBW5EFEFCP2TPEYSS4KAN7TRHJ6XCMVDUUSJ4EB3XBSPQ
Confirmed on round 27892670!
Bob received 2000000 mUSDT!
Signed transaction with txID: 6QZWBHX4TB22UQ4NL2KKZMBAIIBJYELKAVC7VNHK35AW7CCR5VAA
Confirmed on round 27892672!
Carol received 2000000 mUSDT!


## 2. Create Contract

In [3]:
usdc_index = 160363393
usdt_index = 160363405

In [4]:
# Create app and save some $ALGO
from Meson import mesonMainFunc

meson_contract_code = compile_program(algod_client, teal_sentences := compileTeal(
    mesonMainFunc(), Mode.Application, version=8
))
create_app_tx = submit_transaction(alice_private_key, transaction.ApplicationCreateTxn(
    alice_address, sp_func(), on_complete_param, meson_contract_code, blank_program,
    transaction.StateSchema(12, 0), transaction.StateSchema(4, 0),       # TODO: add variable nums
    extra_pages=1,  # Need more pages, or you will get `approval program too long, max len 2048 bytes` error
))
print("Create Meson Contract success! App id: %s, App Address: %s\n" % (
    meson_index := create_app_tx['application-index'],
    meson_address := logic.get_application_address(meson_index)
))

send_amount = 400_000
transfer_algo_to_app_tx = submit_transaction(alice_private_key, transaction.PaymentTxn(
    alice_address, sp_func(), meson_address, send_amount,
))
print("Transfer $ALGO to application address success!")
print("App   {} Balance: {} $ALGO\n".format(meson_address, 
                algod_client.account_info(meson_address).get('amount') / 1e6))

# Add USDC and USDT
submit_transaction(alice_private_key, transaction.ApplicationCallTxn(
    alice_address, sp_func(), meson_index, on_complete_param,
    app_args=['addSupportToken', 1], foreign_assets=[usdc_index]
))
print("Meson App Optin USDC success!\n")
submit_transaction(alice_private_key, transaction.ApplicationCallTxn(
    alice_address, sp_func(), meson_index, on_complete_param,
    app_args=['addSupportToken', 2], foreign_assets=[usdt_index]
))
print("Meson App Optin USDT success!\n")

Signed transaction with txID: EMGFS7GIILT2SAM5OQB6CFF6NL3PNBSM2FAVZ2GWP66VD2V7YM6A
Confirmed on round 28109611!
Create Meson Contract success! App id: 161411332, App Address: 5Y4OAPZDN2DNH7RBIRJIGYDMAAUMRL5RBMLAIYKF7EFE5IVBDZOVTPG2WM

Signed transaction with txID: QNBLKCFPVOKBWYOWX3C2EPP4HKKLNG7LQWQAVGFZCKIPO3FSGOJA
Confirmed on round 28109613!
Transfer $ALGO to application address success!
App   5Y4OAPZDN2DNH7RBIRJIGYDMAAUMRL5RBMLAIYKF7EFE5IVBDZOVTPG2WM Balance: 0.4 $ALGO

Signed transaction with txID: IJWHPNONACAFCE2CW4O76VNDVAFDSZU6TMLC5A5GJ2ZKQ2F5RZTQ
Confirmed on round 28109615!
Meson App Optin USDC success!

Signed transaction with txID: 52RPR4HGEZO2JWSIXDCM6CSBF76QSF5W2OMCCFGKC3ZLP6ZJ4NLA
Confirmed on round 28109617!
Meson App Optin USDT success!



## 3. LP deposit some assets

In [5]:
# LP Opt in Meson
submit_transaction(bob_private_key, transaction.ApplicationOptInTxn(
    bob_address, sp_func(), meson_index
))
print("LP(Bob) opt in Meson App success!")

# LP deposit
amount_deposit = 120 * 1_000_000

submit_transaction_group(bob_private_key, [
    transaction.ApplicationCallTxn(
        bob_address, sp_func(), meson_index, on_complete_param,
        app_args=['deposit', amount_deposit],
        foreign_assets=[usdc_index],
    ), 
    transaction.AssetTransferTxn(
        bob_address, sp_func(), meson_address, amount_deposit, usdc_index,
    )
])
print("LP(Bob) deposit %d mUSDC into Meson App!\n" % (amount_deposit // 1e6))

submit_transaction_group(bob_private_key, [
    transaction.ApplicationCallTxn(
        bob_address, sp_func(), meson_index, on_complete_param,
        app_args=['deposit', amount_deposit],
        foreign_assets=[usdt_index],
    ), 
    transaction.AssetTransferTxn(
        bob_address, sp_func(), meson_address, amount_deposit, usdt_index,
    )
])
print("LP(Bob) deposit %d mUSDT into Meson App!\n" % (amount_deposit // 1e6))

returned = indexer_client.lookup_account_application_local_state(bob_address, application_id=meson_index)
balances_saved = returned['apps-local-states'][0]['key-value']
print("LP(Bob) balance:")
for balance in balances_saved:
    print("Asset %d: %d" % (int.from_bytes(de64(balance['key'])[8:], 'big'), balance['value']['uint'] / 1e6))

Signed transaction with txID: ZNT54ZXT7467RVI6LY6SYBM6JFCK54VG5QDXUKAZLU3XXQNEAJLA
Confirmed on round 28109619!
LP(Bob) opt in Meson App success!
Signed transaction group with gtxID: OEQLC7FRY2UBQOABBAROR4PU3SCKRSNC5OWXO55NKJU44CQDCX4Q
Confirmed on round 28109621!
LP(Bob) deposit 120 mUSDC into Meson App!

Signed transaction group with gtxID: TLY53AJMDPJMQDUDT66ZJ7MRSFPZX5YTS4VVOUZAH6ZJ4BAMCQFA
Confirmed on round 28109623!
LP(Bob) deposit 120 mUSDT into Meson App!

LP(Bob) balance:
Asset 160363393: 120
Asset 160363405: 120


In [ ]:
# LP withdraw
amount_withdraw = 30 * 1_000_000

submit_transaction(bob_private_key, transaction.ApplicationCallTxn(
    bob_address, sp_func(), meson_index, on_complete_param,
    app_args=['withdraw', amount_withdraw, usdt_index],
    foreign_assets=[usdt_index],
))
print("LP(Bob) withdraw %d mUSDT from Meson App!\n" % (amount_withdraw // 1e6))

returned = indexer_client.lookup_account_application_local_state(bob_address, application_id=meson_index)
balances_saved = returned['apps-local-states'][0]['key-value']
print("LP(Bob) balance:")
for balance in balances_saved:
    print("Asset %d: %d" % (int.from_bytes(de64(balance['key'])[8:], 'big'), balance['value']['uint'] / 1e6))

## 4. Start cross-chain swap

### 4.0 Build EncodedSwap and others

In [6]:
from Crypto.Hash import keccak
from web3.auto import w3

# origin format: [bytes32]
# \x01\x00\x01\xe8H\x00\xc0\x00\x00\x00\x00\x00\xe7U& \x00\x00\x00\x00\x00\x00c\xd5\x00B#)\x02\x02\xca
# hex format [uint256, hex64]:
# (0x)010001e84800c00000000000e755262000000000000063d5004223290202ca22
# split values: 0x|01|0001e84800|c00000000000e7552620|0000000000|0063d50042|2329|02|02ca|22
# split variables: 0x|version|amount|salt|fee|expireTs|outChain|outToken|inChain|inToken
# salt[0xc0, 0b11000000] -> [(0)willTransferToContract, (1)feeWaived, -, -, (1)signNonTyped, -, -, -]

def get_expire_ts(delay=90):   # default to 90 minutes
    return int(time.time()) + 60*delay

def build_encoded(amount: int, expireTs: int, outToken, inToken, 
                  salt='c00000000000e7552620', fee='0000000000', return_bytes=True):
    assert amount < 0x1111111111
    version = '01'
    amount_string = hex(amount)[2:].rjust(10, '0')
    expireTs_string = hex(expireTs)[2:].rjust(10, '0')
    outChain = '011b'
    inChain = '011b'
    encoded_string = ''.join([
        '0x', version, amount_string, salt, fee, expireTs_string, outChain, outToken, inChain, inToken
    ])
    return bytes.fromhex(encoded_string[2:]) if return_bytes else encoded_string

def get_swapID(encodedSwap, initiator, return_bytes=True):
    concat = encodedSwap + initiator
    assert len(concat) == 52
    keccak_func = keccak.new(digest_bits=256)
    hash_value = keccak_func.update(concat).hexdigest()
    return bytes.fromhex(hash_value) if return_bytes else hash_value

def keccak256(bytes_str):
    keccak_func = keccak.new(digest_bits=256)
    hash_value = keccak_func.update(bytes.fromhex(bytes_str) if type(bytes_str) == str else bytes_str)
    return hash_value.hexdigest()

request_type = "bytes32 Sign to request a swap on Meson (Testnet)"
release_type = "bytes32 Sign to release a swap on Meson (Testnet)address Recipient"
request_typehash = bytes.fromhex('7b521e60f64ab56ff03ddfb26df49be54b20672b7acfffc1adeb256b554ccb25')
release_typehash = bytes.fromhex('d23291d9d999318ac3ed13f43ac8003d6fbd69a4b532aeec9ffad516010a208c')

amount_transfer = 50 * 1_000_000
phil_private_key = '4719806c5b87c68e046b7b958d4416f66ff752ce60a36d28c0b9c5f29cbc9ab0'
initiator = phil_address = bytes.fromhex('2ef8a51f8ff129dbb874a0efb021702f59c1b211')
encodedSwap = build_encoded(amount_transfer, get_expire_ts(), '02', '01')

# meson_index = 161410247
# meson_address = logic.get_application_address(meson_index)

In [ ]:
# Transfer to contract if needed
send_amount = 400_000
transfer_algo_to_app_tx = submit_transaction(alice_private_key, transaction.PaymentTxn(
    alice_address, sp_func(), meson_address, send_amount,
))
print("Transfer $ALGO to application address success!")
print("App   {} Balance: {} $ALGO\n".format(meson_address, 
                algod_client.account_info(meson_address).get('amount') / 1e6))

### 4.1 postSwap & bondSwap

In [8]:
digest_request = bytes.fromhex(keccak256(request_typehash + bytes.fromhex(keccak256(encodedSwap))))
signed_message = w3.eth.account._sign_hash(digest_request, phil_private_key)
r_int, s_int, v = signed_message.r, signed_message.s, signed_message.v - 27
r, s = int.to_bytes(r_int, 32, 'big'), int.to_bytes(s_int, 32, 'big')

In [9]:
submit_transaction_group(carol_private_key, [
    transaction.ApplicationCallTxn(
        carol_address, sp_func(), meson_index, on_complete_param,
        app_args=['postSwap', encodedSwap, r, s, v, initiator],
        boxes=[(meson_index, encodedSwap)]
    ),
    transaction.AssetTransferTxn(
        carol_address, sp_func(), meson_address, amount_transfer, usdc_index
    ),
    *[transaction.ApplicationCallTxn(
        carol_address, sp_func(), meson_index, on_complete_param,
        app_args=['padding', padding_num]
    ) for padding_num in range(7)],
])
print("Step 1.1. User(Carol) posted swap success!\n")

Signed transaction group with gtxID: BHOJPRCOFUHJR56R6VXZYM5BCEZXGDXYATXZQX3SMOOTZSNYYE6A
Confirmed on round 28109629!
Step 1.1. User(Carol) posted swap success!



In [10]:
submit_transaction(bob_private_key, transaction.ApplicationCallTxn(
    bob_address, sp_func(), meson_index, on_complete_param,
    app_args=['bondSwap', encodedSwap],
    boxes=[(meson_index, encodedSwap)]
))
print("Step 1.2. LP(Bob) Bonded swap success!")

Signed transaction with txID: GXO4UTVQNRP4L4K6YR2DRXM546JD3V7CPKMGEH4IFTGLLAUDD3BQ
Confirmed on round 28109632!
Step 1.2. LP(Bob) Bonded swap success!


In [11]:
print("Meson App boxes (encodedSwap -> postedValue): ")
for name_dict in algod_client.application_boxes(meson_index)['boxes']:
    encoded_key = base64.b64decode(name_dict['name'].encode())
    value_return = algod_client.application_box_by_name(meson_index, encoded_key)
    posted_value = base64.b64decode(value_return['value'])
    if len(posted_value) == 84:
        print('[EncodedSwap] %s, \n\t[PostedValue] (lp, initiator, from_address) \n\t\t\t(%s, \n\t\t\t%s, \n\t\t\t%s)' % (encoded_key.hex(), (p := posted_value.hex())[:64], p[64:104], p[104:168]))

Meson App boxes (encodedSwap -> postedValue): 
[EncodedSwap] 010002faf080c00000000000e755262000000000000064007806011b02011b01, 
	[PostedValue] (lp, initiator, from_address) 
			(1215b7fe9c7e42924ad0ddca31647726769caed8f5f27c701c5112207658c21e, 
			2ef8a51f8ff129dbb874a0efb021702f59c1b211, 
			57f297ed3f9630d0f3437e591c77d7de8f59e1cecc41a0489a4347c5ca0a12f6)


### 4.2 lock

In [15]:
submit_transaction_group(bob_private_key, [
    transaction.ApplicationCallTxn(
        bob_address, sp_func(), meson_index, on_complete_param,
        app_args=['lock', encodedSwap, r, s, v, initiator],
        accounts=[carol_address],
        boxes=[(meson_index, get_swapID(encodedSwap, initiator))]
    ),
    *[transaction.ApplicationCallTxn(
        bob_address, sp_func(), meson_index, on_complete_param,
        app_args=['padding', padding_num]
    ) for padding_num in range(7)],
])
print("Step 2. LP(Bob) lock assets success!\n")

print("Meson App boxes (swapId -> lockedValue): ")
for name_dict in algod_client.application_boxes(meson_index)['boxes']:
    swapid_key = base64.b64decode(name_dict['name'].encode())
    value_return = algod_client.application_box_by_name(meson_index, swapid_key)
    locked_value = base64.b64decode(value_return['value'])
    if len(locked_value) == 69:
        print('[SwapID] %s, \n\t[LockedValue] (lp, until, recipient) \n\t\t\t(%s, \n\t\t\t%s, \n\t\t\t%s)' % (swapid_key.hex(), (p := locked_value.hex())[:64], pd.to_datetime(int(p[64:74], 16) * 1e9), p[74:]))

Signed transaction group with gtxID: ZWREKKMGERDRK5W5RPERXTAYB5YYL3UH2FIXGSBNE3W3HY3UX63Q
Confirmed on round 28109671!
Step 2. LP(Bob) lock assets success!

Meson App boxes (swapId -> lockedValue): 
[SwapID] 20c7c21c1f25bef694bbcefd0d69c6e12e72ad7da54129a7d613314f5f31e1cc, 
	[LockedValue] (lp, until) 
			(1215b7fe9c7e42924ad0ddca31647726769caed8f5f27c701c5112207658c21e, 
			2023-03-02 09:11:24)


### 4.3 release

In [32]:
def decode_algorand_address(algo_addr):
    return base64.b32decode(algo_addr + '======')[:-4]

msgHash = keccak256(encodedSwap + decode_algorand_address(carol_address))
digest_release = bytes.fromhex(keccak256(request_typehash + bytes.fromhex(msgHash)))
signed_message = w3.eth.account._sign_hash(digest_release, phil_private_key)
r_int, s_int, v = signed_message.r, signed_message.s, signed_message.v - 27
r, s = int.to_bytes(r_int, 32, 'big'), int.to_bytes(s_int, 32, 'big')

In [34]:
submit_transaction_group(carol_private_key, [
    transaction.ApplicationCallTxn(
        carol_address, sp_func(), meson_index, on_complete_param,
        app_args=['release', encodedSwap, r, s, v, initiator],
        foreign_assets=[usdt_index],
        boxes=[(meson_index, get_swapID(encodedSwap, initiator))]
    ),
    *[transaction.ApplicationCallTxn(
        carol_address, sp_func(), meson_index, on_complete_param,
        app_args=['padding', padding_num]
    ) for padding_num in range(7)],
])
print("Step 3. User(Carol) release assets success!\n")

print("Meson App boxes (swapId -> lockedValue): ")
for name_dict in algod_client.application_boxes(meson_index)['boxes']:
    swapid_key = base64.b64decode(name_dict['name'].encode())
    value_return = algod_client.application_box_by_name(meson_index, swapid_key)
    locked_value = base64.b64decode(value_return['value'])
    if len(locked_value) == 69:
        print('[SwapID] %s, \n\t[LockedValue] (lp, until, recipient) \n\t\t\t(%s, \n\t\t\t%s, \n\t\t\t%s)' % (swapid_key.hex(), (p := locked_value.hex())[:64], pd.to_datetime(int(p[64:74], 16) * 1e9), p[74:]))

AlgodHTTPError: TransactionPool.Remember: transaction QLUCRRNOERDW6EFGS4BH5X5IN2RNUOSTKQ6S5PHVKHWDDRDYCXHA: logic eval error: assert failed pc=971. Details: pc=971, opcodes=&&
&&
assert


### 4.4 executeSwap

In [34]:
submit_transaction(bob_private_key, transaction.ApplicationCallTxn(
    bob_address, sp_func(), meson_index, on_complete_param,
    app_args=['executeSwap', encodedSwap, 1111, 1111, 0],
    accounts=[carol_address],
    foreign_assets=[usdc_index],
    boxes=[(meson_index, encodedSwap)]
))
print("Step 4. LP(Bob) executeSwap success!\n")

print("Meson App boxes (encodedSwap -> postedValue): ")
for name_dict in algod_client.application_boxes(meson_index)['boxes']:
    encoded_key = base64.b64decode(name_dict['name'].encode())
    value_return = algod_client.application_box_by_name(meson_index, encoded_key)
    posted_value = base64.b64decode(value_return['value'])
    if len(posted_value) == 84:
        print('[EncodedSwap] %s, \n\t[PostedValue] (lp, initiator, from_address) \n\t\t\t(%s, \n\t\t\t%s, \n\t\t\t%s)' % (encoded_key.hex(), (p := posted_value.hex())[:64], p[64:104], p[104:168]))

Signed transaction with txID: 6LFBV6SQCZLFKDEX6A3W4Y3ZVJHUO3TLSLNY5IFQ5D6QQ4CBQEDA
Confirmed on round 27916882!
Step 4. LP(Bob) executeSwap success!

Meson App boxes (encodedSwap -> postedValue): 
[EncodedSwap] 010002faf080c00000000000e755262000000000000063f5cad5011b02011b01, 
	[PostedValue] (lp, initiator, from_address) 
			(1111111111111111111111111111111111111111111111111111111111111111, 
			1111111111111111111111111111111111111111, 
			1111111111111111111111111111111111111111111111111111111111111111)
